# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
# os - for saving and loading files
# json - to work with json files
# math - to round up results
# time - to add a short pause to not overwhelm the server
import os, json, math, time

# to make yelpapi calls
from yelpapi import YelpAPI

# progress bar from tqdm_notebook
from tqdm.notebook import tqdm_notebook

In [2]:
# !pip install yelpapi
# !pip install tqdm

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [3]:
# Load API Credentials
with open('/Users/austi/.secret/yelp_api.json', 'r') as f:
    login = json.load(f)

In [4]:
login.keys()

dict_keys(['client-id', 'api-key'])

In [5]:
#login.items()
#login['API Key']

In [7]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(login['api-key'], timeout_s = 5.0)

### Define Search Terms and File Paths

In [8]:
# set our API call parameters and filename before the first call
location = 'Seattle, WA 98012'
term = 'barbecue'

In [9]:
location.split(',')[1]

' WA 98012'

In [10]:
## Specify folder for saving data
FOLDER = 'Data/'

os.makedirs(FOLDER, exist_ok = True)
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [11]:
JSON_FILE

'Data/Seattle-barbecue.json'

### Check if Json File exists and Create it if it doesn't

In [12]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        #create the folder
        os.makedirs(folder, exist_ok = True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE, 'w') as f:
        json.dump([], f)
        
## If it exists, inform user
else:
    print(f"[ii] {JSON_FILE} already exists.")

[i] Data/Seattle-barbecue.json not found. Saving empty list to file.


### Load JSON FIle and account for previous results

### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [13]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(term = term, location = location)

In [14]:
type(results)

dict

In [15]:
len(results)

3

In [16]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [17]:
results['total']

732

In [18]:
results['region']

{'center': {'longitude': -122.19955444335938, 'latitude': 47.84537662548611}}

In [19]:
results['businesses']

[{'id': 'fU2S1YNmakvTVFC8pIlY9A',
  'alias': 'brileys-bbq-and-grill-lake-forest-park',
  'name': 'Brileys BBQ & Grill',
  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/yKeCFDp2U3V_I2UMCxVGkg/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/brileys-bbq-and-grill-lake-forest-park?adjust_creative=_XtPN5sB9p2gFalQqwrzYQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=_XtPN5sB9p2gFalQqwrzYQ',
  'review_count': 863,
  'categories': [{'alias': 'bbq', 'title': 'Barbeque'},
   {'alias': 'tradamerican', 'title': 'American'}],
  'rating': 4.5,
  'coordinates': {'latitude': 47.73853, 'longitude': -122.29},
  'transactions': ['delivery'],
  'price': '$$',
  'location': {'address1': '15030 Bothell Way NE',
   'address2': '',
   'address3': '',
   'city': 'Lake Forest Park',
   'zip_code': '98155',
   'country': 'US',
   'state': 'WA',
   'display_address': ['15030 Bothell Way NE', 'Lake Forest Park, WA 98155']},
  'phone': '+12064661589',
  'display_phon

In [20]:
## How many results total?
pd.DataFrame(results['businesses'])

id                                    alias  \
0   fU2S1YNmakvTVFC8pIlY9A   brileys-bbq-and-grill-lake-forest-park   
1   f4rnFr_bFWOB44crXfFMbQ                    j-and-l-bbq-snohomish   
2   ZIRJB-lk5cD4L8GUOca3zw                      fat-pig-bbq-edmonds   
3   njvyS8D09NfOhIDbEjNhEg              larrys-smokehouse-snohomish   
4   GAq0SZKYEeWN71hC0FHYrQ                      j-and-l-bbq-everett   
5   D_ClecX3hizRwnQyamtf6g               carolina-smoke-bbq-bothell   
6   6jr4aZ1eMLadC0FntrXKgw           dickeys-barbecue-pit-everett-3   
7   RTgQWV2_XxraHo5b-88kaw                        guam-grub-everett   
8   RmLxbaA32FB0D6l3Rm-TnA            best-of-both-worlds-snohomish   
9   aGyck-ohu3pnwtWHgCEHKQ             wake-n-bacon-bbq-shoreline-2   
10  R7KVJdxrDyDq8_j5ZCJSXA                   moonshine-bbq-lynnwood   
11  dlzTLbbPLwq-J7HRnmSAbg          gabriels-fire-mountlake-terrace   
12  MiKL42aT0Cf5U3n77kUJAw           friendship-foods-bbq-seattle-2   
13  qS8xyE0CYj_qQt1J2warag                     fil-cuisine-lynnwood   
14  a0peClYQHiw5vUPTXl528w      niku-niku-japanese-barbecue-seattle   
15  8zvU5CU6RRQdo8KHmFBXjA                   sweet-radish-everett-3   
16  M2iHLqHGHluSqEIpKYFc1Q  blazing-onion-burger-company-mill-creek   
17  oiotzcCO0paoWK8PmCkVxw           grandmas-in-da-kitchen-everett   
18  E1tPkV_RZyn-ELH8_pVebg            hps-smokehouse-bbq-kirkland-2   
19  vHEoK5y4DFnIK7ly8AhhpA           the-bine-beer-and-food-bothell   

                            name  \
0            Brileys BBQ & Grill   
1                      J & L BBQ   
2                    Fat Pig BBQ   
3             Larry's Smokehouse   
4                      J & L BBQ   
5             Carolina Smoke BBQ   
6          Dickey's Barbecue Pit   
7                      Guam Grub   
8            Best of Both Worlds   
9               Wake N Bacon BBQ   
10                 Moonshine BBQ   
11                Gabriel's Fire   
12          Friendship Foods BBQ   
13                   Fil Cuisine   
14   Niku Niku Japanese Barbecue   
15                  Sweet Radish   
16  Blazing Onion Burger Company   
17       Grandma's In Da Kitchen   
18           HP's Smokehouse BBQ   
19          The Bine Beer & Food   

                                            image_url  is_closed  \
0   https://s3-media3.fl.yelpcdn.com/bphoto/yKeCFD...      False   
1   https://s3-media1.fl.yelpcdn.com/bphoto/2z6_9B...      False   
2   https://s3-media2.fl.yelpcdn.com/bphoto/Xdbg4J...      False   
3   https://s3-media3.fl.yelpcdn.com/bphoto/7R7k4X...      False   
4   https://s3-media3.fl.yelpcdn.com/bphoto/OTdSHc...      False   
5   https://s3-media3.fl.yelpcdn.com/bphoto/NqM0cv...      False   
6   https://s3-media3.fl.yelpcdn.com/bphoto/mSZAuS...      False   
7   https://s3-media2.fl.yelpcdn.com/bphoto/Lvbqkj...      False   
8   https://s3-media4.fl.yelpcdn.com/bphoto/BuzAXU...      False   
9   https://s3-media2.fl.yelpcdn.com/bphoto/sHJ0At...      False   
10  https://s3-media2.fl.yelpcdn.com/bphoto/sTatoc...      False   
11  https://s3-media3.fl.yelpcdn.com/bphoto/oFn74c...      False   
12  https://s3-media1.fl.yelpcdn.com/bphoto/se5c0z...      False   
13  https://s3-media3.fl.yelpcdn.com/bphoto/Iydnwk...      False   
14  https://s3-media1.fl.yelpcdn.com/bphoto/LazyS4...      False   
15  https://s3-media2.fl.yelpcdn.com/bphoto/Ct1Y6a...      False   
16  https://s3-media2.fl.yelpcdn.com/bphoto/Tp-ZtT...      False   
17  https://s3-media4.fl.yelpcdn.com/bphoto/BlxPTk...      False   
18  https://s3-media2.fl.yelpcdn.com/bphoto/m9stK_...      False   
19  https://s3-media4.fl.yelpcdn.com/bphoto/1UrIFC...      False   

                                                  url  review_count  \
0   https://www.yelp.com/biz/brileys-bbq-and-grill...           863   
1   https://www.yelp.com/biz/j-and-l-bbq-snohomish...           300   
2   https://www.yelp.com/biz/fat-pig-bbq-edmonds?a...           289   
3   https://www.yelp.com/biz/larrys-smokehouse-sno...         

- Where is the actual data we want to save?

In [21]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [22]:
(results['total'])/results_per_page

36.6

In [23]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total'])/results_per_page)
n_pages

37

In [24]:
for i in tqdm_notebook(range(1, n_pages+1)):
    ## The block of code we want to TRY to run
    try:

        time.sleep(.2)
        

        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
            
        ## save number of results for to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp.search_query(location = location, term = term, offset = n_results+1)

        ## append new results and save to file
        previous_results.extend(results['businesses'])

        with open(JSON_FILE, 'w') as f:
            json.dump(previous_results, f)

            
    ## What to do if we get an error/exception.
    except Exception as e:
        print(' [!] ERROR', e)


  0%|          | 0/37 [00:00<?, ?it/s]

## Open the Final JSON File with Pandas

In [25]:
df = pd.read_json(JSON_FILE)

In [26]:
df.head()

id                        alias                name  \
0  f4rnFr_bFWOB44crXfFMbQ        j-and-l-bbq-snohomish           J & L BBQ   
1  ZIRJB-lk5cD4L8GUOca3zw          fat-pig-bbq-edmonds         Fat Pig BBQ   
2  njvyS8D09NfOhIDbEjNhEg  larrys-smokehouse-snohomish  Larry's Smokehouse   
3  GAq0SZKYEeWN71hC0FHYrQ          j-and-l-bbq-everett           J & L BBQ   
4  D_ClecX3hizRwnQyamtf6g   carolina-smoke-bbq-bothell  Carolina Smoke BBQ   

                                           image_url  is_closed  \
0  https://s3-media1.fl.yelpcdn.com/bphoto/2z6_9B...      False   
1  https://s3-media2.fl.yelpcdn.com/bphoto/Xdbg4J...      False   
2  https://s3-media3.fl.yelpcdn.com/bphoto/7R7k4X...      False   
3  https://s3-media3.fl.yelpcdn.com/bphoto/OTdSHc...      False   
4  https://s3-media3.fl.yelpcdn.com/bphoto/NqM0cv...      False   

                                                 url  review_count  \
0  https://www.yelp.com/biz/j-and-l-bbq-snohomish...           300   
1  https://www.yelp.com/biz/fat-pig-bbq-edmonds?a...           289   
2  https://www.yelp.com/biz/larrys-smokehouse-sno...           176   
3  https://www.yelp.com/biz/j-and-l-bbq-everett?a...           137   
4  https://www.yelp.com/biz/carolina-smoke-bbq-bo...           484   

                                          categories  rating  \
0            [{'alias': 'bbq', 'title': 'Barbeque'}]     4.5   
1            [{'alias': 'bbq', 'title': 'Barbeque'}]     4.5   
2  [{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...     3.5   
3  [{'alias': 'tradamerican', 'title': 'American'...     4.5   
4            [{'alias': 'bbq', 'title': 'Barbeque'}]     3.5   

                                         coordinates        transactions  \
0    {'latitude': 47.91278, 'longitude': -122.09882}  [pickup, delivery]   
1  {'latitude': 47.8315724, 'longitude': -122.336...  [pickup, delivery]   
2  {'latitude': 47.839407, 'longitude': -122.124891}  [pickup, delivery]   
3    {'latitude': 47.97862, 'longitude': -122.20819}          [delivery]   
4  {'latitude': 47.7784982305512, 'longitude': -1...  [pickup, delivery]   

  price                                           location         phone  \
0    $$  {'address1': '130 Ave D', 'address2': None, 'a...  +13605687222   
1    $$  {'address1': '7533 Olympic View Dr', 'address2...  +14253617640   
2     $  {'address1': '17416 State Route 9', 'address2'...  +14254837001   
3    $$  {'address1': '2915 Colby Ave', 'address2': '',...  +14252593222   
4    $$  {'address1': '24118 Bothell Everett Hwy', 'add...  +14259498672   

    display_phone      distance  
0  (360) 568-7222  10622.755887  
1  (425) 361-7640  10346.304888  
2  (425) 483-7001   5611.595646  
3  (425) 259-3222  14830.981444  
4  (425) 949-8672   7573.454942

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             731 non-null    object 
 1   alias          731 non-null    object 
 2   name           731 non-null    object 
 3   image_url      731 non-null    object 
 4   is_closed      731 non-null    bool   
 5   url            731 non-null    object 
 6   review_count   731 non-null    int64  
 7   categories     731 non-null    object 
 8   rating         731 non-null    float64
 9   coordinates    731 non-null    object 
 10  transactions   731 non-null    object 
 11  price          637 non-null    object 
 12  location       731 non-null    object 
 13  phone          731 non-null    object 
 14  display_phone  731 non-null    object 
 15  distance       731 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 86.5+ KB


In [28]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json', '.csv.gz')
csv_file

'Data/Seattle-barbecue.csv.gz'

In [29]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression = 'gzip', index = False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [30]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 719,557 Bytes
CSV.GZ FILE: 104,463 Bytes
the csv.gz is 6.8881517857997565 times smaller!


## Next Class: Processing the Results and Mapping 